# `Russian Troll Tweets Part-03 NLP pre-processing`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

**`Help Support Channel: Patreon @mrfugudatasci`**
    
**`Or Consider: Buy Me A Coffee @mrfugudatasci`**

**PLEASE READ THIS:**

**`In order to use the CSV, you will need to start from the first notebook due to size of file`**

+ These are sequential videos doing steps to get to this point. You can directly download from Kaggle. I showed how to do this in a previous video. [Kaggle api](https://youtu.be/mmxbyfhG6Ss)

In [4]:
import pandas as pd
import numpy as np
import re

# plotting image with word cloud
from PIL import Image                    # for our images
import numpy as np                       # convert image to np arrays
from wordcloud import WordCloud,ImageColorGenerator,STOPWORDS # generate word cloud
import matplotlib.pyplot as plt          # plotting
import seaborn as sns

# nlp
import nltk                              # text processing
from nltk import word_tokenize           # split sentence into list of words
from nltk.corpus import stopwords        # remove: and,it,i,etc
import string                            # remove punctuation
from collections import defaultdict      # dictionary with values as lists
from nltk.stem import WordNetLemmatizer  # remove word endings etc

%matplotlib inline

In [5]:
troll_02=pd.read_csv('troll_tweets_vid02.csv')

troll_02=troll_02.drop(['content','Unnamed: 0'],axis = 1)

troll_02.tail()

,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http
2973365,_YOUR_LIFESTYLE,Unknown,Uzbek,9/7/2015 1:31,60,4,412,Other_Tweet,Russian,0,NonEnglish,Санкционный пинг-понг
2973366,_YOUR_LIFESTYLE,Unknown,Russian,9/8/2015 14:49,60,4,416,Other_Tweet,Russian,0,NonEnglish,Горизонтальный кальян NanoSmoke
2973367,_YOUR_LIFESTYLE,Unknown,Russian,9/9/2015 18:52,60,4,418,Other_Tweet,Russian,0,NonEnglish,Фотограф Kai-Hendrik Schroeder
2973368,_YOUR_LIFESTYLE,Unknown,Russian,9/9/2015 22:26,60,4,419,Other_Tweet,Russian,0,NonEnglish,К чему стремится компания Netflix (США)
2973369,_YOUR_LIFESTYLE,Unknown,Russian,9/9/2015 8:17,60,4,417,Other_Tweet,Russian,0,NonEnglish,В Петербурге появилось вегетарианское кафе в к...


# We need to do some formatting of these data for NLP!

+ Depending on our analysis dictates what we do...

In [6]:
round(troll_02['retweet'].mean(),3)

0.441

In [7]:
# Convert str to date/time with pandas
troll_02['publish_date']=pd.to_datetime(troll_02['publish_date'],
                                        infer_datetime_format=True)

troll_02.sort_values('publish_date',ascending=True)

# Change formating to US dates
troll_02['publish_date']=troll_02['publish_date'].\
dt.strftime('%m/%d/%Y')


In [8]:
troll_02['publish_date']=pd.to_datetime(troll_02['publish_date'],
                                        infer_datetime_format=True)

# get range to encapsulate before and after election 2016
mask = (troll_02['publish_date'] > '08/31/2016') &\
(troll_02['publish_date']  <= '01/01/02017')


In [110]:

troll_dt_rng=troll_02[troll_02["publish_date"].isin(pd.date_range('2016-10-01', '2017-01-01'))].\
sort_values('publish_date')

In [111]:
troll_02.loc[7,'content without http']

'@SenatorMenendez @CarmenYulinCruz Doesnt matter that CNN doesnt report on your crimes. This wont change the fact that youre going down.'

# `Create Hashtag and Retweet columns:`

We have some simple solutions but there are issues:
This works but provides us with only first occurence `troll_02['content without http'].str.extract(r'@(\S+)')`

If we need a to find more: use `.extractall()` unfortunately it returns multi-index table and difficult to use.

Therefore, I used a loop to deal with this. Unless, you have a good work around beaware of limitations as well as your options.

In [112]:
# Use this to parse Hashtags and @user

def tweet_col_regx(df_col,re_pat):
    q=[]
    for i in df_col:
        if type(i)==str and re.findall(re_pat,i):
            q.append(re.findall(re_pat,i))
        else:
            q.append('None')
    return q

In [113]:
tweet_ref=tweet_col_regx(troll_dt_rng['content without http'],r'@(\S+)')

troll_dt_rng['tweet_ref']=tweet_ref

In [114]:

hashtag_ref=tweet_col_regx(troll_dt_rng['content without http'],r'#(\S+)')

troll_dt_rng['hashtag_ref']=hashtag_ref

In [115]:
# troll_dt_rng[troll_dt_rng['tweet_ref']!='None'] #verify it is working in this column

In [119]:
# convert author/user name to lowercase for comparisons
troll_dt_rng['author']=troll_dt_rng['author'].map(lambda x: x.lower())


In [124]:

troll_dt_rng.head(9)


user_list = troll_dt_rng['author'].unique()

#Define a function to test whether a value is part
# of the authors (for apply function below)
def test_in(el):
    return (el in user_list)

#create a new column in tweets indicating whether the source 
# of a retweeted tweet was in our list of user_names

troll_dt_rng[troll_dt_rng['tweet_ref'].apply(test_in)==True].tail()

/Users/zatoichi59/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if __name__ == '__main__':


,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http,tweet_ref,hashtag_ref,follower_to_following_Ratio
2670625,todayinsyria,United Kingdom,English,2016-12-28,9397,28748,11341,Other_Tweet,news,0,NewsFeed,@todayinsyria its unknown whether he was suppl...,[todayinsyria],[ISIS],3.059274
2670682,todayinsyria,United Kingdom,English,2016-12-29,7888,28798,11376,Other_Tweet,news,0,NewsFeed,@todayinsyria more than 20 fighters from both ...,[todayinsyria],None,3.650862
2670663,todayinsyria,United Kingdom,English,2016-12-29,7888,28813,11392,Other_Tweet,news,0,NewsFeed,@todayinsyria Iraqi security forces have liber...,[todayinsyria],[Mosul],3.652764
2665847,todayinsyria,United Kingdom,English,2017-01-01,7391,28783,11419,Other_Tweet,news,0,NewsFeed,@todayinsyria #ISIS releases pictures of bombe...,[todayinsyria],[ISIS],3.894331
2665850,todayinsyria,United Kingdom,English,2017-01-01,7390,28779,11422,Other_Tweet,news,0,NewsFeed,@todayinsyria #SAA artillery pounded #ISIS mil...,[todayinsyria],"[SAA, ISIS]",3.894317


In [121]:
# Friend To Follower Ratios: 
troll_dt_rng['follower_to_following_Ratio']=np.where(troll_dt_rng['followers']/\
                        troll_dt_rng['following']==np.inf
                     , troll_dt_rng['followers'], 
         troll_dt_rng['followers']/troll_dt_rng['following'])


print('Proportion of Users w/ High Follower Count (ALL-SUBset data):',\
round(troll_dt_rng[troll_dt_rng['follower_to_following_Ratio']>2].shape[0]/\
troll_dt_rng[troll_dt_rng['follower_to_following_Ratio']<2].shape[0],3))

Proportion of Users w/ High Follower Count (ALL-SUBset data): 0.358


In [122]:
troll_dt_rng.sort_values('follower_to_following_Ratio',ascending=False)[:10]

,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http,tweet_ref,hashtag_ref,follower_to_following_Ratio
1487417,kadirovrussia,Unknown,Russian,2016-12-07,0,95629,11191,Other_Tweet,Russian,0,NonEnglish,Лично договорился?,None,None,95629.0
1429375,jenn_abrams,United States,English,2016-12-07,0,63581,21385,QUOTE_TWEET,Right,1,RightTroll,"Guys, I got blocked, something is wrong with T...",None,None,63581.0
1429376,jenn_abrams,United States,English,2016-12-07,0,63580,21386,Other_Tweet,Right,0,RightTroll,@Trump_Row @PatriotTrumpet ✋�,"[Trump_Row, PatriotTrumpet]",None,63580.0
1429374,jenn_abrams,United States,English,2016-12-07,0,63578,21383,Other_Tweet,Right,0,RightTroll,@SandyGraves7 @NYCdeb8tr Im not a trump supporter,"[SandyGraves7, NYCdeb8tr]",None,63578.0
1944157,novostiekb,Unknown,Russian,2016-11-29,0,45230,9209,Other_Tweet,Russian,0,NonEnglish,В Екатеринбурге открылся голубиный ресторан,None,None,45230.0
1969071,novostiomska,Unknown,Russian,2016-11-14,0,40273,7392,Other_Tweet,Russian,0,NonEnglish,Обещанный к 300-летию Омска кинотеатр Первомай...,None,[Омск],40273.0
686822,crystal1johnson,United States,English,2016-12-08,0,25443,3091,Other_Tweet,left,0,LeftTroll,"My job during my season is to provide cheer, j...",None,None,25443.0
686823,crystal1johnson,United States,English,2016-12-08,0,25443,3092,QUOTE_TWEET,left,1,LeftTroll,Black Santa is the greatest!,None,None,25443.0
384865,boeing_is_back,Malaysia,Russian,2016-10-17,0,24235,3036,Other_Tweet,Russian,0,NonEnglish,"Все, что вы хотели знать о сыре, но боялись сп...",None,None,24235.0
1938044,novostidamask,Unknown,Russian,2016-12-06,0,21463,10342,Other_Tweet,Russian,0,NonEnglish,Российские военные доставили гуманитарную помо...,None,None,21463.0


# `Check fraction of tweets as retweets:`

In [125]:
# Subset English: 

eng_subset=troll_dt_rng[troll_dt_rng['language']=='English']
eng_subset.head()

,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http,tweet_ref,hashtag_ref,follower_to_following_Ratio
2299654,richmondvoice,United States,English,2016-10-01,4731,2811,4778,Other_Tweet,local,0,NewsFeed,Chesterfield County Police respond to a wareho...,None,None,0.594166
2299655,richmondvoice,United States,English,2016-10-01,4731,2811,4779,Other_Tweet,local,0,NewsFeed,Petersburg Police investigating after deadly s...,None,None,0.594166
2299656,richmondvoice,United States,English,2016-10-01,4731,2811,4781,Other_Tweet,local,0,NewsFeed,Flint Residents On Federal Aid Deal: Too Littl...,None,None,0.594166
2299657,richmondvoice,United States,English,2016-10-01,4731,2811,4782,Other_Tweet,local,0,NewsFeed,Mob informant trades witness protection for $1...,None,None,0.594166
2299658,richmondvoice,United States,English,2016-10-01,4731,2811,4783,Other_Tweet,local,0,NewsFeed,Man found dead outside Petersburg home,None,None,0.594166


In [126]:
# Subset Russian:

russ_subset=troll_dt_rng[troll_dt_rng['language']=='Russian']
russ_subset.head()

,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http,tweet_ref,hashtag_ref,follower_to_following_Ratio
2857939,whitehousecards,United States,Russian,2016-10-01,3974,7927,722,Other_Tweet,Russian,0,NonEnglish,На Newsweek были совершены кибератаки после то...,None,None,1.994716
1980409,novostispb,Russian Federation,Russian,2016-10-01,8732,105448,38355,Other_Tweet,Russian,0,NonEnglish,"Ёжик, блуждая в тумане, забрёл в #Петербург. К...",None,[Петербург.],12.076042
1980408,novostispb,Russian Federation,Russian,2016-10-01,8731,105447,38353,Other_Tweet,Russian,0,NonEnglish,Так выглядела Сенная площадь в далеком 1933 году,None,None,12.077311
1980407,novostispb,Russian Federation,Russian,2016-10-01,8731,105447,38352,Other_Tweet,Russian,0,NonEnglish,Очень актуальная инструкция для Петербурга - к...,None,None,12.077311
1980406,novostispb,Russian Federation,Russian,2016-10-01,8731,105448,38350,Other_Tweet,Russian,0,NonEnglish,Друзья! Предлагаем Вам небольшую задачку � И н...,None,None,12.077425


In [127]:
#english:
print('Subset Proportion Retweets:',round(troll_dt_rng['retweet'].mean(),3))
print('-----------------------')
print('English Subset fraction Retweets:',round(eng_subset['retweet'].mean(),3))
print('-----------------------')
print('Russian Subset fraction Retweets:',round(russ_subset['retweet'].mean(),3))

Subset Proportion Retweets: 0.723
-----------------------
English Subset fraction Retweets: 0.738
-----------------------
Russian Subset fraction Retweets: 0.675


# `Citations & Help:`

# ◔̯◔

https://www.kaggle.com/auntiebody/coordinating-influence-on-twitter (very good notebook)

https://www.kaggle.com/laurenliz22/russian-troll-tweets (other motivation)